In [1]:

from pathlib import Path

import glob2
import datetime

import numpy as np
from tqdm.notebook import tqdm
import tensorflow as tf

from utils.data_reading.catalogs.ISC import ISC_file
from utils.data_reading.sound_data.station import StationsCatalog
from utils.physics.bathymetry.bathymetry_grid import BathymetryGrid
from utils.physics.sound.sound_model import HomogeneousSoundModel
from utils.physics.sound.sound_velocity_grid import MonthlySoundVelocityGridOptimized
from utils.transformations.features_extractor import STFTFeaturesExtractor

print(tf.config.list_physical_devices())

2024-09-25 15:59:21.629218: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-25 15:59:21.635459: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-25 15:59:21.750318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 15:59:21.750408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 15:59:21.753237: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2024-09-25 15:59:24.903514: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-25 15:59:24.905231: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
datasets_yaml = "/home/plerolland/Bureau/dataset.yaml"

year = 2018
isc_file = f"/home/plerolland/Bureau/catalogs/ISC/eqk_isc_{year}.txt"
isc = ISC_file(isc_file)

sound_model_h = HomogeneousSoundModel()
sound_model_g = MonthlySoundVelocityGridOptimized([f"../../data/sound_model/min-velocities_month-{i:02d}.nc" for i in range(1,13)], interpolate=True)
bathy_model = BathymetryGrid.create_from_NetCDF("../../data/geo/GEBCO_2023_sub_ice_topo.nc", lat_bounds=[-75, 35], lon_bounds=[-20, 180])

stations_c = StationsCatalog(datasets_yaml).filter_out_undated().filter_out_unlocated()
stft_computer = STFTFeaturesExtractor(None, vmin=60, vmax=140, f_min=5, f_max=60)

# output
detections_file = f"../../data/detections/{year}/detections.npy"

In [58]:
# filter isc events
to_del = set()
for ID, event in isc.items.items():
    if bathy_model.get_nearest_values(event.get_pos()) > 0:
        to_del.add(ID)
    if isc[ID].get_pos()[0]>-5 and isc[ID].get_pos()[1] > 115 or isc[ID].get_pos()[0]>-30 and isc[ID].get_pos()[1] > 130 or isc[ID].get_pos()[0]>-45 and isc[ID].get_pos()[1] > 170\
            or isc[ID].get_pos()[0]>-20 and isc[ID].get_pos()[1] > 85:
        to_del.add(ID)
for ID in to_del:
    del isc.items[ID]
print(f"{len(to_del)} terrestrial events removed from catalog")

0 terrestrial events removed from catalog


### Detections computation

In [5]:
if not Path(detections_file).is_file():
    print("starting detection")
    to_read = glob2.glob(f"../../data/detections/{year}/*.csv")
    
    detections = {}
    for file in to_read:
        with open(file, "r") as f:
            lines = f.readlines()
        lines = [line.strip().split(",") for line in lines if len(line.strip())>0]
        lines = np.array([[datetime.datetime.strptime(line[0], "%Y%m%d_%H%M%S"), float(line[1]), line[2]] for line in lines])
        lines[:,2] = [np.array(l.split(";")).astype(np.float32) for l in lines[:,2]]
        lines = lines[np.argsort(lines[:,0])]
    
        # merge close detections
        to_del = set()
        i, j = 0, 0
        while j<len(lines):
            delta = (np.abs(lines[i,0] - lines[j,0]))
            if delta.total_seconds() < 5:
                lines[i,0] += delta / 2
                to_del.add(j)
            else:
                i = j
            j+=1
        lines = np.delete(lines, list(to_del), axis=0)
        
        
        station_name = "-".join(file.split("_")[-1].split("-")[:-1])
        station_y = int(file.split("_")[-1].split("-")[-1][:-4])
        station = stations_c.by_names(station_name).by_starting_year(station_y)[0]
        
        print(f"{len(list(to_del))} lines deleted ; {len(lines)} left for station {station}")
        
        detections[station] = lines
    np.save(detections_file, detections)

In [6]:
detections = np.load(detections_file, allow_pickle=True).item()

In [7]:
# get inter-station distances
basic_sound_model = HomogeneousSoundModel()
stations = list(detections.keys())
d_h = {s1: {s2: datetime.timedelta(seconds=0) for s2 in stations} for s1 in stations}
for s1 in stations:
    for s2 in stations:
        d_h[s1][s2] = datetime.timedelta(seconds=basic_sound_model.get_sound_travel_time(s1.get_pos(), s2.get_pos()))
        print(f"{s1}-{s2} : {d_h[s1][s2]}")

# merge all detections and sort them by date
merged_detections = []
for s, dets in detections.items():
    for det in dets:
        merged_detections.append((det[0], det[1], tuple(det[2]), s))
merged_detections = np.array(merged_detections, dtype=np.object_)
merged_detections = merged_detections[np.argsort(merged_detections[:,0])]

station_ELAN_2018-station_ELAN_2018 : 0:00:00
station_ELAN_2018-station_MADE_2018 : 0:40:20.120202
station_ELAN_2018-station_MADW_2018 : 0:35:09.529054
station_ELAN_2018-station_NEAMS_2018 : 0:35:48.664488
station_ELAN_2018-station_RTJ_2018 : 0:41:01.621956
station_ELAN_2018-station_SSEIR_2018 : 0:29:30.562140
station_ELAN_2018-station_SSWIR_2018 : 0:23:56.077778
station_ELAN_2018-station_SWAMS-bot_2018 : 0:19:18.915028
station_ELAN_2018-station_WKER2_2018 : 0:12:29.692215
station_ELAN_2018-station_MADE_2017 : 0:40:20.778781
station_ELAN_2018-station_MADW_2017 : 0:35:09.229006
station_ELAN_2018-station_NEAMS_2017 : 0:35:48.721658
station_ELAN_2018-station_SSEIR_2017 : 0:29:30.782122
station_ELAN_2018-station_SWAMS_2017 : 0:19:19.125930
station_ELAN_2018-station_WKER2_2017 : 0:12:29.668607
station_ELAN_2018-station_H01W1_2018 : 0:50:45.217739
station_ELAN_2018-station_H04N1_2018 : 0:15:34.284465
station_ELAN_2018-station_H04S1_2018 : 0:14:46.804349
station_ELAN_2018-station_H08S1_2018 :

### Study of clustering

In [8]:
make_study = False

In [59]:
max_allowed_delta = 50

# compute distances of close in time events, expected run time ~2 min for 10,000 events with max_allowed_delta = 50 days
idx_last_valid = 0
delta_km = {}
delta_d = {}
delta = {}
IDs = list(isc.items.keys())
for i, (ID, isc_event) in tqdm(enumerate(isc.items.items()), total=len(IDs)):
    if i==0:
        continue
    date, pos = isc_event.date, isc_event.get_pos()
    
    # peremption of clusters (by date)
    while idx_last_valid < i and isc[IDs[idx_last_valid]].date < date - datetime.timedelta(days=max_allowed_delta):
        idx_last_valid += 1
        
    dates, poss = [], []
    for idx in range(idx_last_valid, i):
        poss.append(isc[IDs[idx]].get_pos())
        dates.append(isc[IDs[idx]].date)
    dates, poss = np.array(dates), np.array(poss)
    # approximation of 1) loxodromy and 2) constant deg to km conversion ; acceptable because small distances and "far" from pole
    deltas_km = np.sqrt((poss[:,0]-pos[0])**2+(poss[:,1]-pos[1])**2)*111
    deltas_d = np.array([d.total_seconds() / 86400 for d in np.abs(dates - date)])
    deltas = np.sqrt(deltas_km ** 2 + deltas_d ** 2)
    
    for idx in range(idx_last_valid, i):
        ID_ = IDs[idx]
        delta_km.setdefault(ID, {})[ID_] = deltas_km[idx-idx_last_valid]
        delta_d.setdefault(ID, {})[ID_] = deltas_d[idx-idx_last_valid]
        delta.setdefault(ID, {})[ID_] = deltas[idx-idx_last_valid]

# make it symmetric, expected run time ~ 40 s for above conditions
for ID in IDs:
    for ID_ in delta.setdefault(ID, {}).keys():
        delta_km.setdefault(ID_, {})[ID] = delta_km[ID][ID_]
        delta_d.setdefault(ID_, {})[ID] = delta_d[ID][ID_]
        delta.setdefault(ID_, {})[ID] = delta[ID][ID_]

  0%|          | 0/1467 [00:00<?, ?it/s]

In [10]:
# expected run time of 30 min for above conditions
def get_valid(allowed_delta, delta, IDs):
    valid = {ID: {} for ID in IDs}
    for ID in tqdm(IDs, leave=False):
        for ID_ in IDs:
            if (ID_ in delta[ID] and delta[ID][ID_] < allowed_delta) or ID == ID_:
                valid[ID][ID_] = True
    return valid

if make_study:
    allowed_deltas_to_try = list(range(1, max_allowed_delta))
    valid = {a_d: {ID: {} for ID in IDs} for a_d in allowed_deltas_to_try}
    for allowed_delta in tqdm(allowed_deltas_to_try):
        valid = get_valid(allowed_delta, delta, IDs)

In [11]:
def add_to_cluster(current_clusters, ID_to_cluster, ID, cluster_idx):
    current_clusters[cluster_idx].append(ID)
    ID_to_cluster[ID] = cluster_idx

def merge_clusters(current_clusters, ID_to_cluster, cluster_1_idx, cluster_2_idx):
    for ID in current_clusters[cluster_2_idx]:
        add_to_cluster(current_clusters, ID_to_cluster, ID, cluster_1_idx)
    del current_clusters[cluster_2_idx]
    
def get_clusters(IDs, valid):
    clusters = {}
    ID_to_cluster = {}
    for ID in tqdm(IDs, leave=False):
        if ID not in ID_to_cluster:
            new_cluster_key = np.max(list(clusters.keys())+[0])+1 # + [0] to handle empty list case
            clusters[new_cluster_key] = []
            add_to_cluster(clusters, ID_to_cluster, ID, new_cluster_key)
        
        for ID_ in valid[ID].keys():
            if ID_ in ID_to_cluster:
                if ID in ID_to_cluster and ID_to_cluster[ID] != ID_to_cluster[ID_]:
                    merge_clusters(clusters, ID_to_cluster, ID_to_cluster[ID], ID_to_cluster[ID_])
            else:
                add_to_cluster(clusters, ID_to_cluster, ID_, ID_to_cluster[ID])         
    return clusters

if make_study:
    clusters = {}
    for allowed_delta in tqdm(allowed_deltas_to_try):
        clusters[allowed_delta] = get_clusters(IDs, valid[allowed_delta])

In [12]:
if make_study:
    med_nb, med_delta, med_delta_km, med_delta_d, max_nb, max_delta, max_delta_km, max_delta_d = {}, {}, {}, {}, {}, {}, {}, {}
    
    for a_d, current_clusters in tqdm(clusters.items()):
        nb = []
        
        d, d_km, d_d = [], [], []
        for cluster in tqdm(current_clusters.values(), leave=False):
            nb.append(len(cluster))
            d_, d_km_, d_d_ = [], [], []
            if len(cluster) > 1:
                for ID in cluster:
                    for ID_ in cluster:
                        if ID_ not in delta[ID]:  # they were too distant away (in time) to have a delta
                            pos1, pos2, date1, date2 = isc[ID].get_pos(), isc[ID_].get_pos(), isc[ID].date, isc[ID_].date
                            delta_km[ID][ID_] = np.sqrt((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)*111
                            delta_km[ID_][ID] = delta_km[ID][ID_]
                            delta_d[ID][ID_] = np.abs(date1-date2).total_seconds() / 86400
                            delta_d[ID_][ID] = delta_d[ID][ID_]
                            delta[ID][ID_] = np.sqrt(delta_km[ID][ID_] ** 2 + delta_d[ID][ID_] ** 2)
                            delta[ID_][ID] = delta[ID][ID_]
                        d_.append(delta[ID][ID_])
                        d_km_.append(delta_km[ID][ID_])
                        d_d_.append(delta_d[ID][ID_])
                d.append(np.max(d_))
                d_km.append(np.max(d_km_))
                d_d.append(np.max(d_d_))
        med_nb[a_d] = np.median(nb)
        med_delta[a_d] = np.median(d)
        med_delta_km[a_d] = np.median(d_km)
        med_delta_d[a_d] = np.median(d_d)
        max_nb[a_d] = np.max(nb)
        max_delta[a_d] = np.max(d)
        max_delta_km[a_d] = np.max(d_km)
        max_delta_d[a_d] = np.max(d_d)

In [13]:
from matplotlib import pyplot as plt

if make_study:
    plt.scatter(list(med_nb.keys()), list(med_nb.values()), label="med nb of events per cluster")
    plt.scatter(list(max_nb.keys()), list(max_nb.values()), label="max nb of events per cluster")
    plt.legend()
    plt.xlabel("max allowed delta")
    plt.ylabel("nb of events per cluster")

In [14]:
if make_study:
    plt.scatter(list(med_delta.keys()), list(med_delta.values()), label="med delta")
    plt.scatter(list(med_delta_km.keys()), list(med_delta_km.values()), label="med delta km")
    plt.scatter(list(med_delta_d.keys()), list(med_delta_d.values()), label="med delta d")
    plt.legend()
    plt.xlabel("max allowed delta")
    plt.ylabel("km or days")

In [15]:
if make_study:
    plt.scatter(list(max_delta.keys()), list(max_delta.values()), label="max delta")
    plt.scatter(list(max_delta_km.keys()), list(max_delta_km.values()), label="max delta km")
    plt.scatter(list(max_delta_d.keys()), list(max_delta_d.values()), label="max delta d")
    plt.legend()
    plt.xlabel("max allowed delta")
    plt.ylabel("km or days")

In [65]:
allowed_delta = 30
valid_ = get_valid(allowed_delta, delta, IDs)
clusters_ = get_clusters(IDs, valid_)

  0%|          | 0/1467 [00:00<?, ?it/s]

  0%|          | 0/1467 [00:00<?, ?it/s]

In [66]:
res = ""
for id, cluster in clusters_.items():
    if len(cluster)>1:
        for ID in cluster:
            res += f"{isc[ID].get_pos()[0]},{isc[ID].get_pos()[1]},{id}\n"
with open("res.csv", "w") as f:
    f.write(res)
    
date_min = [np.min([isc[ID].date for ID in cluster]) for cluster in clusters_.values()]
clusters_ = list(clusters_.values())
clusters_ = [clusters_[i] for i in np.argsort(date_min)]

In [67]:
MIN_PROBA = 0.4
merged_detections_kept = merged_detections[merged_detections[:,1]>MIN_PROBA]
print(f"{len(merged_detections_kept)} detections kept out of {len(merged_detections)}")

473199 detections kept out of 5229255


In [97]:
def find_candidates(detections, detection, d_h, seen_dates, tolerance, min_p=0):
    date, embedding, s1 = detection[0], detection[2], detection[-1]
    if date in seen_dates:
        return []
    candidates = []
    for s2 in detections.keys():
        candidates.append([])
        detections_s2 = detections[s2]
        idx_min = np.searchsorted(detections[s2][:,0], date + d_h[s1][s2] - tolerance, side='right')
        idx = idx_min  # note : iterating like this instead of finding max with searchsorted is way more efficent
        while idx < len(detections[s2]) and detections[s2][idx,0] < date + d_h[s1][s2] + tolerance:
            det = detections_s2[idx]
            if det[0] not in seen_dates and det[1]>min_p:
                delta_embedding = np.sqrt(np.sum(np.square(det[2] - embedding))) / np.sqrt(16)
                if det[0] not in seen_dates:
                    candidates[-1].append((det[0], det[1], delta_embedding, s2))
            idx += 1
        if len(candidates[-1]) == 0:
            candidates = candidates[:-1]
        else:
            candidates[-1] = np.array(candidates[-1])
    return candidates

def best_matchup(scores, nb, to_skip=0):
    scores_merged = []
    for i in range(len(scores)):
        for j, s in enumerate(scores[i]):
            scores_merged.append((i, j, s))

    scores_merged.sort(key=lambda x: x[2])
    selected = []
    chosen_idxs = set()
    idx = 0
    n_skipped = 0
    while len(selected) != nb:
        i, j, s = scores_merged[idx]
        if i not in chosen_idxs:
            if len(selected) == nb - 1:
                if n_skipped < to_skip:
                    n_skipped += 1
                    continue
            selected.append((i, j))
            chosen_idxs.add(i)
        idx += 1
    return selected

In [ ]:
import copy
from associator import locate

TOLERANCE = datetime.timedelta(seconds=50)
NB = 4
RES_FILE = "matchups_clusters.csv"
MIN_PROBA_MATES = 0.2

for cluster in tqdm(clusters_):
    # 1 ) update expected time shift between hydrophones
    # 2 ) define allowed deviation from this estimate
    # 3 ) look for first event or two matching this (high expectation of nb of stations ?), using time bounds of +/-selected_allowed_delta days
    # 4 ) use the time shift of this event as a new baseline
    # 5 ) detect all events
    if len(cluster) > 1:
        centroid = np.mean([isc[ID].get_pos() for ID in cluster], axis=0)
        dates = [isc[ID].date for ID in cluster]
        date_min, date_max = np.min(dates), np.max(dates)
        date_mid = date_min + (date_max - date_min)/2
        
        expected = {s: sound_model_h.get_sound_travel_time(centroid, s.get_pos(), date=date_mid) for s in stations}
        # d_h[s1][s2] = given a detection on s1, time to wait before getting it on s2 (can be negative)
        d_h = {s1: {s2: datetime.timedelta(seconds=expected[s2]-expected[s1]) for s2 in stations} for s1 in stations}
        
        # in bounds search! (inter-search)
        idx_detections, seen_dates = {s: 0 for s in stations}, set()
        idx_min = np.searchsorted(merged_detections_kept[:,0], date_min, side='right')
        idx_max = np.searchsorted(merged_detections_kept[:,0], date_max, side='left')
        print(idx_min, idx_max)
        for idx in tqdm(range(idx_min, idx_max), leave=False):
            detection = merged_detections_kept[idx]
            candidates = find_candidates(detections, detection, d_h, seen_dates=seen_dates, tolerance=TOLERANCE, min_p=MIN_PROBA_MATES)
            #print(len(candidates))
            if len(candidates) < NB:
                continue
            scores = [c[:, 2] for c in candidates]
            best_m = best_matchup(scores, nb=NB)
            
            matchup = [detection] + [candidates[i][j] for i, j in best_m]
            loc_res = locate(matchup, sound_model_h, 500)
    
            # in case it worked, go further
            if loc_res:
                loc_res = locate(matchup, sound_model_g, 300, initial_pos=loc_res.x[1:])
                if loc_res:
                    det_times = [date] + [c[0] for c in matchup]
                    date_event = np.min(det_times) + datetime.timedelta(seconds=loc_res.x[0])
    
                    # add other stations
                    changed = False
                    loc_res_updated = copy.deepcopy(loc_res)
                    matchup_updated = copy.deepcopy(matchup)
    
                    seen_stations = set([c[-1] for c in matchup])
                    for c in candidates:
                        if c[0][-1] in seen_stations:
                            continue
                        expected_time = date_event + datetime.timedelta(
                            seconds=sound_model_h.get_sound_travel_time(c[0][-1].get_pos(), loc_res_updated.x[1:]))
                        found = False
                        for chosen in c:
                            if np.abs(chosen[0] - expected_time).total_seconds() < 10:
                                found = True
                                break
                        if not found:
                            continue
                        matchup_updated = matchup_updated + [chosen]
                        loc_res_new = locate(matchup_updated, sound_model_h, 500)
                        if loc_res_new:  # update location
                            changed = True
                            seen_stations.add(chosen[-1])
                            loc_res_updated = loc_res_new
                        else:  # rollback
                            matchup_updated = matchup_updated[:-1]
    
                    if changed:
                        loc_res_new = locate(matchup_updated, sound_model_g, 300, initial_pos=loc_res_updated.x[1:])
                        if loc_res_new:
                            loc_res = loc_res_new
                            matchup = matchup_updated
                    if loc_res:
                        det_times = [date] + [c[0] for c in matchup]
                        for d in det_times:
                            seen_dates.add(d)  # to avoid using the same detection twice
                        date_event = np.min(det_times) + datetime.timedelta(seconds=loc_res.x[0])
                        try:
                            J = loc_res.jac
                            cov = np.linalg.inv(J.T.dot(J))
                            var = np.sqrt(np.diagonal(cov))
                        except:
                            var = [-1, -1, -1]
    
                        to_write = f'{date_event.strftime("%Y%m%d_%H%M%S")},{loc_res.x[1]:.4f},{loc_res.x[2]:.4f},{var[0]:.4f},{var[1]:.4f},{var[2]:.4f}'
                        for d, _, _, s in matchup:
                            to_write += f',{s.name}-{s.date_start.year},{d.strftime("%Y%m%d_%H%M%S")}'
                        with open(RES_FILE, "a") as f:
                            f.write(to_write + "\n")

  0%|          | 0/776 [00:00<?, ?it/s]

4211 16335


  0%|          | 0/12124 [00:00<?, ?it/s]